In [1]:
# Used to plot figure 2

In [2]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import scipy.stats
from sklearn.linear_model import Ridge

In [ ]:
# Parameters
files = [f"/eos/home-a/amgruber/SWAN_projects/XSec_ROOT/TTreeMerges/proj{i+1}_merged.root" for i in range(58)]
tree_name = "FlatTree_VARS"
branch_name = "Enu_true"
n_bins = 40
bin_edges = np.linspace(0, 8000, n_bins + 1)  # 800 bins from 0 to 8000 MeV

# Initialize a numpy array to hold the histograms
histograms = np.zeros((n_bins, len(files)))

# Loop over the files and fill the histograms
for idx, file in enumerate(tqdm(files, desc="Processing ROOT files")):
    with uproot.open(file) as root_file:
        tree = root_file[tree_name]
        data = tree[branch_name].array(library="np")
        
        # Filling the histogram for the current file
        hist, _ = np.histogram(data, bins=bin_edges)
        histograms[:, idx] = hist

# Display the shape of the final histogram array
print(f"Histogram array shape: {histograms.shape}")

# Example visualization of the first histogram
plt.plot(bin_edges[:-1], histograms[:, 0])
plt.xlabel("Energy (MeV)")
plt.ylabel("Counts")
plt.title("Histogram of First ROOT File")
plt.grid(True)
plt.show()


In [ ]:
norms = np.loadtxt("/eos/home-a/amgruber/SWAN_projects/FluxTest/norms.txt") # Replace with a .txt file with events per year according to event plan (can be generated with EventRatesAccordingToRunPlan.ipynb)
# Verify dimensions
assert histograms.shape[1] == len(norms), "Normalization list length does not match number of histograms."

# Normalize each histogram to its corresponding norm
normalized_histograms = histograms * (norms / np.sum(histograms, axis=0, keepdims=True))

In [ ]:
# Parameters
flux_file = "Fluxes.ND.root"
tree_name = "LBNF_numu_flux"
target_loc = 1.  # Center of the target flux in GeV (adjust accordingly)
target_scale = 0.07  # Width of the target flux in GeV
model = Ridge  # Ridge = Tikhonov; Can be set to None for OLS
alpha = 1e-12  # Regularization strength for Tikhonov

# Get DUNE near detector muon neutrino flux as a function of off-axis angle
with uproot.open(flux_file) as fFluxes:
    oa_flux = fFluxes[tree_name].values()
    energy_bins = fFluxes[tree_name].axis(0).edges()  # in GeV
    angle_bins = fFluxes[tree_name].axis(1).edges()   # in milliradians

energy_bin_centers = np.add(energy_bins[:-1], energy_bins[1:]) / 2.

# Define target flux (Gaussian)
rescaling = oa_flux.max()
target_flux = [scipy.stats.norm.pdf(x, loc=target_loc, scale=target_scale) * rescaling for x in energy_bin_centers]
target_flux = np.array(target_flux)

# ========== OLS ==========
x_ols, residuals, rank, s = np.linalg.lstsq(oa_flux, target_flux, rcond=None)
predicted_ols = oa_flux @ x_ols

# ========== Tikhonov ==========
lm = model(alpha=alpha, fit_intercept=False)
lm.fit(oa_flux, target_flux)
x_ridge = lm.coef_
predicted_ridge = oa_flux @ x_ridge

In [ ]:
# Apply OLS and Tikhonov coefficients
event_rate_ols = normalized_histograms @ x_ols
event_rate_ridge = normalized_histograms @ x_ridge

# === Step 1: Build the statistical uncertainty array ===
uncertainties = np.sqrt(normalized_histograms)

# === Step 2: Add uncertainties in quadrature according to the coefficients ===
ols_uncertainty = np.sqrt((uncertainties ** 2) @ (x_ols ** 2))
ridge_uncertainty = np.sqrt((uncertainties ** 2) @ (x_ridge ** 2))

# Scaling factor
scale_factor = 1000

# Apply the scaling
event_rate_ols /= scale_factor
ols_uncertainty /= scale_factor
event_rate_ridge /= scale_factor
ridge_uncertainty /= scale_factor

In [ ]:
plt.rcParams['text.usetex'] = True
fonts = 28

# Define the bin edges (in GeV)
bin_edges = np.linspace(0, 8, n_bins + 1)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

# === Plotting Event Rates with Uncertainties ===
plt.figure(figsize=(8, 6), dpi=300)  # Adjusted for two-column layout

# Plot Ridge with uncertainties first (it will be in the background)
plt.errorbar(bin_centers, event_rate_ols, yerr=ols_uncertainty, 
             fmt='o', markersize=5, label="OLS", color='#1f77b4', capsize=4, zorder=1)

# Plot OLS with uncertainties second (it will be on top)
plt.errorbar(bin_centers, event_rate_ridge, yerr=ridge_uncertainty, 
             fmt='o', markersize=5, label="Tikhonov", color='#ff7f0e', capsize=4, zorder=2)

# Labels and grid
plt.xlabel(r"$E_{\nu}\ [\mathrm{GeV}]$", fontsize=fonts)
plt.ylabel(r"$\frac{d\tilde{N}}{dE_{\nu}}\ \mathrm{[a.u.]}$", fontsize=fonts)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(fontsize=fonts - 2)
plt.xticks(fontsize=fonts)
plt.yticks(fontsize=fonts)
plt.xlim([0,8])
plt.tight_layout()

# Save the figure in high resolution
plt.savefig('event_rate_comparison_twocolumn.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
plt.figure(figsize=(8, 6), dpi=300)  # Adjusted for two-column layout

angles = [i for i in range(58)]

# Plot the line markers
plt.plot(angles, x_ols, '-o', label='OLS', color='#1f77b4', markersize=5, linewidth=2)
plt.plot(angles, x_ridge, '-o', label='Tikhonov', color='#ff7f0e', markersize=5, linewidth=2)

# Labels and grid
plt.xlabel(r"Off-Axis Position Index", fontsize=fonts)
plt.ylabel(r"Coefficient Value", fontsize=fonts)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(fontsize=fonts - 2)
plt.xticks(fontsize=fonts)
plt.yticks(fontsize=fonts)
plt.xlim([-0.5,57.5])
plt.tight_layout()

# Save the figure in high resolution
plt.savefig('coefficients_comparison_lineplot_twocolumn.png', dpi=300, bbox_inches='tight')
plt.show()